# DESIGN PROJECT

## Mise au point d’un système de caméra stéréoscopique pour l’estimation du débit des rivières

In [2]:
import numpy as np
import cv2
import datetime
import camera_matrix

#<add code to run ffmpeg command via ssh>
cap0 = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/vid1_dx.MP4")
cap1 = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/vid1_sx.MP4")
fps = cap0.get(cv2.CAP_PROP_FPS)
mtx, newcameramtx, dist, w, h = camera_matrix()
while cap0.isOpened():
    # Capture frame-by-frame
    ret0, frame0 = cap0.read()
    bw0= cv2.cvtColor(frame0, cv2.COLOR_BGR2GRAY)
    mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
    dst0= cv2.remap(bw0, mapx, mapy, cv2.INTER_LINEAR)
    if ret0== True :
        cv2.imshow('video', dst0)
        #img.convert('L')
        if cv2.waitKey(int(1000/fps)) == ord("q"):
            break
    else:
        break
cap0.release()

cv2.destroyAllWindows()
# import useful packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

TypeError: 'module' object is not callable

In [9]:
frame0.type()

AttributeError: 'NoneType' object has no attribute 'type'